In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
%cd /content/drive/MyDrive/ML Mid Term Project

/content/drive/MyDrive/ML Mid Term Project


In [ ]:
%%shell

wget https://www.cnts.ua.ac.be/conll2000/chunking/train.txt.gz
gzip -d train.txt.gz

--2023-09-29 16:44:42--  https://www.cnts.ua.ac.be/conll2000/chunking/train.txt.gz
Resolving www.cnts.ua.ac.be (www.cnts.ua.ac.be)... 146.175.13.81
Connecting to www.cnts.ua.ac.be (www.cnts.ua.ac.be)|146.175.13.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 611540 (597K) [application/x-gzip]
Saving to: ‘train.txt.gz.1’

train.txt.gz.1      100%[===================>] 597.21K  1.08MB/s    in 0.5s    

2023-09-29 16:44:44 (1.08 MB/s) - ‘train.txt.gz.1’ saved [611540/611540]



In [ ]:
%%shell

cut -d ' ' -f 1,2 train.txt > train_processed.txt

In [ ]:
def remove_empty_lines(filepath):
    with open(filepath, 'r') as file:
        lines = file.readlines()

    # Filter out the empty lines
    cleaned_lines = [line for line in lines if line.strip()]

    # Write cleaned lines back to the same file
    with open(filepath, 'w') as file:
        file.writelines(cleaned_lines)




In [ ]:
# Define your file path
filepath = '/content/drive/MyDrive/ML Mid Term Project/test_data.txt'

# Remove empty lines and update the same file
remove_empty_lines(filepath)

In [ ]:
# Define your file path
filepath = '/content/drive/MyDrive/ML Mid Term Project/act.txt'

# Remove empty lines and update the same file
remove_empty_lines(filepath)

In [ ]:

# Define your file path
filepath = '/content/drive/MyDrive/ML Mid Term Project/unLabTest.txt'

# Remove empty lines and update the same file
remove_empty_lines(filepath)

In [ ]:
import random
from sklearn.model_selection import train_test_split

# Define the input and output file paths
input_file = '/content/drive/MyDrive/ML Mid Term Project/train_processed.txt'
output_train_file = 'train_set.txt'
output_dev_file = 'dev_set.txt'

# Read the data from the input file
with open(input_file, 'r') as f:
    data = f.read()

# Split the data by newline characters to get individual data points
data_points = data.strip().split('\n')

# Shuffle the data points randomly
random.shuffle(data_points)

# Split the data into training (80%) and dev (20%) sets
train_data, dev_data = train_test_split(data_points, test_size=0.2, random_state=42)

# Save the training and dev sets to separate files
with open(output_train_file, 'w') as f:
    f.write('\n'.join(train_data))

with open(output_dev_file, 'w') as f:
    f.write('\n'.join(dev_data))


In [45]:
import os
import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack


In [51]:
def format_data(fname, include_y=True):
    sentences = []
    with open(fname, 'r') as f:
        sentence = []
        for line in f:
            if not line.strip():
                if sentence:
                    sentences.append(sentence)
                    sentence = []
            else:
                word, tag = line.strip().split()
                sentence.append((word, tag) if include_y else word)
        if sentence:
            sentences.append(sentence)
    return sentences


In [56]:
def untag(tagged_sentence):
    words = []
    for item in tagged_sentence:
        try:
            w, _ = item
            words.append(w)
        except ValueError:
            print("Problematic entry:", item)
    return words


In [48]:
def features(sentence, index):
    word = sentence[index]
    prefix = lambda s, n: s[:n] if len(s) > n else s
    suffix = lambda s, n: s[-n:] if len(s) > n else s

    return {
        'nb_terms': len(sentence),
        'word': word,
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'is_capitalized': word[0].upper() == word[0] if word else False,
        'is_all_caps': word.upper() == word if word else False,
        'is_all_lower': word.lower() == word if word else False,
          'prefix-1': prefix(word, 1),
          'prefix-2': prefix(word, 2),
          'prefix-3': prefix(word, 3),
          'prefix-4': prefix(word, 4),
          'prefix-5': prefix(word, 5),
          'suffix-1': suffix(word, 1),
          'suffix-2': suffix(word, 2),
          'suffix-3': suffix(word, 3),
          'suffix-4': suffix(word, 4),
          'suffix-5': suffix(word, 5),
          'i-1_prefix-3': prefix(sentence[index-1], 3) if index > 0 else '',
          'i-1_suffix-3': suffix(sentence[index-1], 3) if index > 0 else '',
          'i+1_prefix-3': prefix(sentence[index+1], 3) if index < len(sentence) - 1 else '',
          'i+1_suffix-3': suffix(sentence[index+1], 3) if index < len(sentence) - 1 else '',
        'prev_word': sentence[index - 1] if index > 0 else '',
        'next_word': sentence[index + 1] if index < len(sentence) - 1 else '',
        'has_hyphen': '-' in word,
        'is_numeric': word.isdigit(),
        'capitals_inside': word[1:].lower() != word[1:]
    }


In [58]:
# Combine handcrafted and vectorized features
def combine_features(matrix_hc, matrix_vect):
    diff = matrix_hc.shape[0] - matrix_vect.shape[0]
    if diff > 0:
        matrix_hc = matrix_hc[:matrix_vect.shape[0], :]
    else:
        matrix_vect = matrix_vect[:matrix_hc.shape[0], :]
    return hstack([matrix_hc, matrix_vect])

In [49]:
def transform_to_dataset(tagged_sentences):
    X, y = [], []
    for sentences in tagged_sentences:
        for index in range(len(sentences)):
            X.append(features(untag(sentences), index))
            y.append(sentences[index][1])
    return X, y

In [69]:
from scipy.sparse import hstack

def combine_features(matrix1, matrix2):
    """
    Combines two feature matrices.

    Parameters:
    - matrix1: First feature matrix.
    - matrix2: Second feature matrix.

    Returns:
    - Combined feature matrix.
    """
    if matrix1.shape[0] != matrix2.shape[0]:
        raise ValueError(f"The matrices have mismatched row counts: {matrix1.shape[0]} vs {matrix2.shape[0]}")
    return hstack([matrix1, matrix2])

In [83]:
def split_sentences(tagged_data):
    end_punctuations = [".", "!", "?"]
    sentences = []
    current_sentence = []
    for word, tag in tagged_data:
        current_sentence.append((word, tag))
        if word in end_punctuations:
            sentences.append(current_sentence)
            current_sentence = []
    # Add the last sentence if it doesn't end with punctuation
    if current_sentence:
        sentences.append(current_sentence)
    return sentences

split_train_sentences = split_sentences(train_sentences[0])

split_dev_sentences = split_sentences(dev_sentences[0])

In [52]:
train_path = "/content/drive/MyDrive/ML Mid Term Project/train_set.txt"
dev_path = "/content/drive/MyDrive/ML Mid Term Project/dev_set.txt"
train_sentences = format_data(train_path)
dev_sentences = format_data(dev_path)

In [64]:
print(train_sentences[:5])

[[('something', 'NN'), ('As', 'IN'), ('politics', 'NNS'), ('offset', 'VB'), ('days', 'NNS'), ('to', 'TO'), ('bank', 'NN'), (',', ','), ('thinks', 'VBZ'), ('member', 'NN'), ('debut', 'NN'), ('health', 'NN'), ('GNP', 'NNP'), ('stable', 'JJ'), (',', ','), ('But', 'CC'), ('points', 'NNS'), ('short-term', 'JJ'), ('Yankus', 'NNP'), (',', ','), ('certain', 'RB'), ('produce', 'VB'), ('see', 'VB'), (',', ','), ('Steinhardt', 'NNP'), ('sought', 'VBD'), ('country', 'NN'), ('flier', 'NN'), ('few', 'JJ'), ('see', 'VB'), ('to', 'TO'), ('``', '``'), ('will', 'MD'), ('to', 'TO'), ('of', 'IN'), ('premises', 'NNS'), ('million', 'CD'), ('the', 'DT'), ('equipment', 'NN'), ('yield', 'VB'), ('.', '.'), ('BankAmerica', 'NNP'), ('government', 'NN'), ('the', 'DT'), ('of', 'IN'), ('#', '#'), ('alive', 'JJ'), ('Little', 'NNP'), ('company', 'NN'), ('month', 'NN'), ("'s", 'POS'), (',', ','), ('stock', 'NN'), ('enlist', 'VB'), ('water', 'NN'), (',', ','), ('close', 'VB'), ('issued', 'VBN'), ('computer', 'NN'), ('se

In [65]:
print(dev_sentences[:5])

[[('overall', 'JJ'), ('the', 'DT'), ('thrift', 'NN'), ('J.', 'NNP'), ('atoms', 'NNS'), ('Politburo', 'NNP'), ('Anderson', 'NNP'), ('34', 'CD'), (',', ','), ('has', 'VBZ'), ('lot', 'NN'), ('are', 'VBP'), ('will', 'MD'), ('expect', 'VBP'), ('for', 'IN'), ('receipts', 'NNS'), ('Exchange', 'NNP'), ('federal-systems', 'JJ'), ('The', 'DT'), ('earnings', 'NNS'), ('it', 'PRP'), ('older', 'JJR'), ('equal', 'JJ'), ('takeover', 'NN'), ('million', 'CD'), ('the', 'DT'), ('equivalent', 'NN'), ('raised', 'VBN'), ("''", "''"), ('then', 'RB'), ('have', 'VBP'), ('to', 'TO'), ('a', 'DT'), ('2.4', 'CD'), ('Mr.', 'NNP'), ('an', 'DT'), ('campaigning', 'VBG'), ('he', 'PRP'), ('is', 'VBZ'), ('in', 'IN'), ('both', 'DT'), ('.', '.'), ('venture-capital', 'JJ'), ('.', '.'), ('deputy', 'NN'), ('for', 'IN'), ('call', 'VB'), ('and', 'CC'), ("'s", 'POS'), ('York', 'NNP'), ('years', 'NNS'), ('at', 'IN'), ('that', 'IN'), ('-RRB-', ')'), ('company', 'NN'), ('surface', 'NN'), ('a', 'DT'), ('tax', 'NN'), ('ad', 'NN'), ('t

In [ ]:
# Transform the training and development sentences
X_train_hc, y_train_hc = transform_to_dataset(train_sentences)
X_dev_hc, y_dev_hc = transform_to_dataset(dev_sentences)

# Save the transformed datasets using joblib
joblib.dump(X_train_hc, 'X_train.pkl')
joblib.dump(y_train_hc, 'y_train.pkl')
joblib.dump(X_dev_hc, 'X_dev.pkl')
joblib.dump(y_dev_hc, 'y_dev.pkl')

In [53]:
X_train_hc = joblib.load('X_train.pkl')
y_train_hc = joblib.load('y_train.pkl')
X_dev_hc = joblib.load('X_dev.pkl')
y_dev_hc = joblib.load('y_dev.pkl')

In [54]:
# Feature Hashing for handcrafted features
hasher = FeatureHasher(input_type='dict')
X_train_hc_vect = hasher.transform(X_train_hc)
X_dev_hc_vect = hasher.transform(X_dev_hc)

In [87]:
# Vectorization using CountVectorizer and TfidfVectorizer
count_vect = CountVectorizer()
tfidf_vect = TfidfVectorizer()

# Convert each word in train and dev datasets into "documents"
plain_train_words = [word for sentence in train_sentences for word, _ in sentence]
plain_dev_words = [word for sentence in dev_sentences for word, _ in sentence]

# Vectorization
X_train_word_count_vect = count_vect.fit_transform(plain_train_words)
X_train_word_tfidf_vect = tfidf_vect.fit_transform(plain_train_words)

X_dev_word_count_vect = count_vect.transform(plain_dev_words)
X_dev_word_tfidf_vect = tfidf_vect.transform(plain_dev_words)

# Stack the vectorized matrices horizontally
X_train_word_vect = hstack([X_train_word_count_vect, X_train_word_tfidf_vect])
X_dev_word_vect = hstack([X_dev_word_count_vect, X_dev_word_tfidf_vect])




In [85]:
print(X_train_hc_vect.shape)
print(X_train_vect.shape)


(169427, 1048576)
(7069, 26966)


In [88]:
X_train_combined = combine_features(X_train_hc_vect, X_train_word_vect)
X_dev_combined = combine_features(X_dev_hc_vect, X_dev_word_vect)


In [89]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train_hc)
y_dev = label_encoder.transform(y_dev_hc)

In [90]:
lr = LogisticRegression(C=1, solver='liblinear', multi_class='auto', random_state=42)
lr.fit(X_train_combined, y_train)


LogisticRegression(C=1, random_state=42, solver='liblinear')

In [91]:
dev_score = lr.score(X_dev_combined, y_dev)
print("Dev score : ", dev_score)

lr_pred = lr.predict(X_dev_combined)
accuracy = accuracy_score(y_dev, lr_pred)
f1 = f1_score(y_dev, lr_pred, average='macro')

print("Dev accuracy: {:.2f}%".format(accuracy * 100))
print("F1 score: {:.2f}".format(f1))

Dev score :  0.9393144208037825
Dev accuracy: 93.93%
F1 score: 0.84


In [92]:
# Load the unlabeled data
def load_unlabeled_data(filepath):
    with open(filepath, 'r') as file:
        words = [line.strip() for line in file]
    return words

unlabeled_words = load_unlabeled_data('/content/drive/MyDrive/ML Mid Term Project/test_data.txt')

# Convert the words into the feature format
unlabeled_features = [features(unlabeled_words, index) for index in range(len(unlabeled_words))]

# Vectorize the features and combine with the hash vectors
unlabeled_features_vect = hasher.transform(unlabeled_features)
unlabeled_combined = hstack([unlabeled_features_vect, count_vect.transform(unlabeled_words), tfidf_vect.transform(unlabeled_words)])

# Predict the POS tags
predicted_tags = lr.predict(unlabeled_combined)
decoded_tags = label_encoder.inverse_transform(predicted_tags)

# Write the words and their predicted tags to a new file
with open('predictedTags1.txt', 'w') as file:
    for word, tag in zip(unlabeled_words, decoded_tags):
        file.write(f"{word} {tag}\n")


In [93]:
from sklearn.metrics import accuracy_score, f1_score

def read_file(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f.readlines():
            line = line.strip()
            if line:  # Check for non-empty lines
                components = line.split()
                if len(components) != 2:
                    print(f"Skipped problematic line: {line}")
                    continue
                word, label = components
                data.append((word, label))
    return data

def compare_files(predicted_file_path, actual_file_path):
    predicted_data = read_file(predicted_file_path)
    actual_data = read_file(actual_file_path)

    y_pred = [label for word, label in predicted_data]
    y_true = [label for word, label in actual_data]

    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')

    return accuracy, f1

predicted_file = '/content/drive/MyDrive/ML Mid Term Project/predictedTags1.txt'
actual_file = '/content/drive/MyDrive/ML Mid Term Project/act.txt'

accuracy, f1 = compare_files(predicted_file, actual_file)
print(f'Accuracy: {accuracy:.2f}%')
print(f'F1 Score: {f1:.2f}')

Accuracy: 0.92%
F1 Score: 0.92
